In [3]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [5]:
print(tf.__version__)

2.16.1


In [6]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url, untar=True, cache_dir='.', cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")

84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


In [10]:
os.listdir(dataset_dir)

['imdbEr.txt', 'test', 'imdb.vocab', 'README', 'train']

In [13]:
train_dir = os.path.join(dataset_dir, "train")
os.listdir(train_dir)

['urls_unsup.txt',
 'neg',
 'urls_pos.txt',
 'unsup',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

In [17]:
print("--- negative review example ---")
sample_file = os.path.join(train_dir, 'neg/1_1.txt')
with open(sample_file) as f:
  print(f.read())

print()

print("--- positive review example ---")
sample_file = os.path.join(train_dir, 'pos/0_9.txt')
with open(sample_file) as f:
  print(f.read())

--- negative review example ---
Robert DeNiro plays the most unbelievably intelligent illiterate of all time. This movie is so wasteful of talent, it is truly disgusting. The script is unbelievable. The dialog is unbelievable. Jane Fonda's character is a caricature of herself, and not a funny one. The movie moves at a snail's pace, is photographed in an ill-advised manner, and is insufferably preachy. It also plugs in every cliche in the book. Swoozie Kurtz is excellent in a supporting role, but so what?<br /><br />Equally annoying is this new IMDB rule of requiring ten lines for every review. When a movie is this worthless, it doesn't require ten lines of text to let other readers know that it is a waste of time and tape. Avoid this movie.

--- positive review example ---
Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much c

In [62]:
remove_dir = os.path.join(train_dir, "unsup")
shutil.rmtree(remove_dir)

FileNotFoundError: [Errno 2] No such file or directory: './aclImdb/train/unsup'

In [89]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
print(dataset)
for element in dataset:
  print(element)

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


2024-05-19 21:06:17.326278: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [81]:
batch_size = 32
seed = 42

raw_train_dataset = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    labels="inferred",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=seed
)

raw_train_dataset

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [138]:
dataset = tf.data.Dataset.range(10)

print(dataset)

for i in dataset.take(4):
    print(i.numpy())

list(dataset.as_numpy_iterator())


<_RangeDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
0
1
2
3


2024-05-19 21:14:19.860587: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-19 21:14:19.864219: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [139]:
for text_batch, label_batch in raw_train_dataset.take(1):
    for i in range(3):
        print("REVIEW", text_batch.numpy()[i])
        print("LABEL", label_batch.numpy()[i])

REVIEW b'There is so much that is wrong with this film, but to sum it up: Terrible acting- so bad it must have been on purpose. poor script - they may have had some good ideas but this was not the best way to present the story. ridiculously bad ending- in some cases the ending manages to save the film-not in this case. if you manage to sit through the entire film you will want to kick yourself at the end because the ending is not even worth waiting for. This is the worst film i have seen in a long time. It was complete torture sitting through this film, i would have appreciated someone warning me in advance. So do yourself a favor. Watch this film only if you have absolutely nothing better to do. Even then you will regret having put yourself through the unspeakable torture.'
LABEL 0
REVIEW b'\'What I Like About You\' is definitely a show that I couldn\'t wait to see each day. Amanda Bynes is such an excellent actress and I grew up watching her show: \'The Amanda Show.\' She\'s a very f

2024-05-19 21:16:26.620976: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [141]:
print("LABEL 0:", raw_train_dataset.class_names[0])
print("LABEL 1:", raw_train_dataset.class_names[1])

LABEL 0: neg
LABEL 1: pos


In [147]:
raw_validation_dataset = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=seed
)

raw_validation_dataset

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [149]:
raw_test_dataset = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)

raw_test_dataset

Found 25000 files belonging to 2 classes.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [176]:
def standardize_html(text: str) -> str:
    lowercase = tf.strings.lower(text)
    stripped_text = tf.strings.regex_replace(lowercase, "<br />", " ")
    
    return tf.strings.regex_replace(stripped_text, "[%s]" % re.escape(string.punctuation), "")

In [177]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    standardize=standardize_html,
    output_mode="int",
    output_sequence_length=sequence_length
)

vectorize_layer

<TextVectorization name=text_vectorization_3, built=False>

In [180]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_dataset.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

2024-05-19 22:30:21.557494: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [181]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [182]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_dataset))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_dataset.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b"First and foremost I would like to say that I'm a huge Sarah Silverman fan, and having other people say that and then rag on the pilot is beyond me. Everything in the pilot was in typical Sarah Silverman form, maybe not directly funny, but the situation and the delivery are what counts.<br /><br />If you liked Jesus is Magic then I don't see how you wouldn't like this. It has that same flow and that same rhythm. True it's only for the true fans, but if you are you'll be pleased.<br /><br />Again only for the true fans, there's no way around that. If you're not used to her style then you wouldn't get why this is funny. However, it is, and I hope you think so too.", shape=(), dtype=string)
Label pos
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[  83,    3, 6968,   10,   59,   38,    6,  131,   12,  141,    4,
         625, 2697, 7457,  325,    3,  253,   78,   79,  131,   12,    3,
          92,    1,   20,    2, 1800,    7,  706,   69,  27

In [183]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  silent
 313 --->  night
Vocabulary size: 10000


In [184]:
train_dataset = raw_train_dataset.map(vectorize_text)
validation_dataset = raw_validation_dataset.map(vectorize_text)
test_dataset = raw_test_dataset.map(vectorize_text)